# Climate Data Studio
## Getting started with the Sust Global API

In [1]:
import requests
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, shape, Polygon
from IPython.display import display, HTML
import contextily as ctx
import matplotlib.pyplot as plt
import folium, matplotlib

display_df = lambda df: display(HTML(df.to_html()))

API_KEY = "y22BmlNQLBr1AwSH5Yej"
PORTFOLIO = "us-counties"
PROJECT = "cds-demo"

endpoints = {
    "portfolios": f"https://explorer.sustglobal.io/api/portfolios/?api_key={API_KEY}&project={PROJECT}",
    "assets"    : f"https://explorer.sustglobal.io/api/portfolios/{PORTFOLIO}/assets?api_key={API_KEY}&project={PROJECT}&rows=200",
    "summary"   : f"https://explorer.sustglobal.io/api/portfolios/{PORTFOLIO}/datasets/physical/summary?api_key={API_KEY}&project={PROJECT}&rows=200",
    "physical"  : f"https://explorer.sustglobal.io/api/portfolios/{PORTFOLIO}/datasets/physical/items?api_key={API_KEY}&project={PROJECT}&rows=200"
}

explore_defaults = {
    "tiles": ctx.providers.Esri.WorldGrayCanvas,
    "marker_kwds": {
        "radius": 5
    }
}

def get_endpoint_as_df(endpoint_id, params={}, fetch_args={"print_pages": False}):
    
    def fetch_with_pagination(endpoint_id, **kwargs):
        page = 1
        buff = []
        if endpoint_id == "portfolios":
            return pd.DataFrame(requests.get(f'{endpoints[endpoint_id]}').json())
        while True:
            response = requests.get(f'{endpoints[endpoint_id]}&page={page}', params=params).json()
            if len(response) > 0:
                if kwargs["print_pages"]:
                    print(page)
                buff = buff + response
                page += 1
            else:
                return pd.DataFrame(buff)
            
    res_df = fetch_with_pagination(endpoint_id, **fetch_args)
    if endpoint_id == "assets":
        if "geometry" in res_df.columns: # Polygons
            return gpd.GeoDataFrame(res_df.copy().drop("geometry", axis=1), geometry=res_df.geometry.apply(shape), crs="EPSG:4326")
        else: # Points
            return gpd.GeoDataFrame(res_df.copy(), geometry=gpd.points_from_xy(res_df.lng, res_df.lat), crs="EPSG:4326")
    elif endpoint_id == "physical":
        return pd.concat([
            res_df.drop('risk_exposure', axis=1),
            pd.DataFrame(list(res_df.risk_exposure))
        ], axis=1)
    else:
        return res_df
    
    
def map_risk_summary_json_to_single_hazard(summary_series, hazard):
    return pd.concat([
        summary_series.drop("risk_summaries", axis=1), 
        pd.DataFrame(list(summary_series.risk_summaries.apply(lambda s: [i for i in s if i["hazard"] == hazard][0])))
    ], axis=1)


def expand_timeseries_json_to_df(timeseries):
    return pd.DataFrame(list(timeseries))


def get_top_n(df, col, n=10, **kwargs):
    return df.sort_values(col, **kwargs).iloc[0:n].drop("geometry", axis=1)


    

In [2]:
portfolios = get_endpoint_as_df("portfolios")
display_df(portfolios)

,portfolio_id,portfolio_name,created_at,updated_at,status
0,022256aede96a409,all_national_forests,2022-07-25T21:22:34Z,2022-07-25T21:31:00Z,Risk data available
1,7440a84ba0101c12,global-mangrove-watch,2022-07-20T21:32:55Z,2022-07-20T22:11:50Z,Risk data available
2,f37fb494bb33d5e7,us-counties,2022-07-18T23:57:21Z,2022-07-19T21:15:01Z,Risk data available


In [ ]:
gdf_assets = get_endpoint_as_df("assets")
gdf_assets.geometry = gdf_assets.geometry.simplify(0.01)
gdf_assets.explore(**explore_defaults)

In [ ]:
summaries_15_year = get_endpoint_as_df("summary", params={'window': 15, 'scenario': 'ssp585'})
# display_df(summaries)


In [ ]:
display_df(summaries_15_year)

## Plotting Wildfire Potential

In [ ]:
summary = map_risk_summary_json_to_single_hazard(summaries_15_year, "wildfire")
assets_combined_summaries = gdf_assets.merge(summary, on="portfolio_index")
assets_combined_summaries.explore(column='risk_score', cmap='YlOrRd', **explore_defaults)


## Get Top Wildfire Geographies

In [ ]:
display_df(get_top_n(assets_combined_summaries, "risk_score", ascending=False))

## Plotting Water Stress

In [ ]:
summary = map_risk_summary_json_to_single_hazard(summaries_15_year, "water_stress")
assets_combined_summaries = gdf_assets.merge(summary, on="portfolio_index")
assets_combined_summaries.explore(column='risk_score', cmap='YlOrRd', **explore_defaults)

In [ ]:
display_df(get_top_n(assets_combined_summaries, "risk_score", ascending=False))

In [ ]:
exposure_timeseries = get_endpoint_as_df("physical", params={
    'hazard': 'wildfire', 'indicator': 'unified_prob', 'scenario': 'ssp585', 'measure': 'mid'
})

In [ ]:
exposure_timeseries